In [23]:
import mysql.connector  as connection
import pandas as pd
import numpy as np
import datetime as dt
import os
import re

In [24]:
file_dir = 'C:/Users/User/Desktop/Moje dokumenty/Raporty obce/Kacper/63_issue_158946_payments/Excel/2022/'
db_credentials_dir = 'C:/Users/User/Desktop/Moje dokumenty/Raporty obce/Kacper/63_issue_158946_payments/'
sql_dir = 'C:/Users/User/AppData/Roaming/DBeaverData/workspace6/General/Scripts/'

In [25]:
Beliani_de = pd.read_excel(file_dir + 'Full List v2.xlsx', sheet_name='White')

In [26]:
Beliani_de['Text'] = Beliani_de['Text'].str.lower()

In [27]:
Beliani_de['Transaction_Date'] = pd.to_datetime(Beliani_de['Transaction_Date'])

In [28]:
# Run MySQL query to fetch auftrags
# Putty must be running while running the script
with open(db_credentials_dir + 'db_credentials.txt', 'r') as f:
    db_credentials = f.read().splitlines()

try:
    db_connections = connection.connect(
        host=db_credentials[0],
        user=db_credentials[1],
        password=db_credentials[2],
        database=db_credentials[3]
    )
    sql_file = open(sql_dir + 'Script-19.sql')
    sql_query = sql_file.read()
    auctions = pd.read_sql(sql_query,db_connections)

    # close the connection
    db_connections.close()  
except Exception as e:
    db_connections.close()
    print(str(e))

C:\Users\User\AppData\Local\Temp\ipykernel_14072\2843205706.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  auctions = pd.read_sql(sql_query,db_connections)


In [29]:
# Clean auctions data
auctions['auction_number'] = auctions['auction_number'].astype(str)
auctions['auction_number'] = auctions['auction_number'].astype(str).replace(r'\.0', '', regex=True)
auctions['invoice_number'] = auctions['invoice_number'].astype(str)
auctions['invoice_number'] = auctions['invoice_number'].astype(str).replace(r'\.0', '', regex=True)

In [13]:
#Beliani_invoice = Beliani_de[Beliani_de['Text'].str.contains('credit', na=False)]

In [30]:
Beliani_de_auctions = Beliani_de[Beliani_de['Text'].str.contains('auction', na=False)]
Beliani_de_tickets = Beliani_de[Beliani_de['Text'].str.contains('ticket', na=False)]

In [34]:
# to filter out ticket numbers with space
in_books_22_split = Beliani_de_tickets
in_books_22_split['ticket_number'] = Beliani_de_tickets['Text'].str.extract(r'(\d+)')
in_books_22_split = in_books_22_split.dropna(subset=["ticket_number"]) 

C:\Users\User\AppData\Local\Temp\ipykernel_14072\600884060.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_books_22_split['ticket_number'] = Beliani_de_tickets['Text'].str.extract(r'(\d+)')


In [36]:
# to filter out ticket numbers without space
in_books_22_comb = Beliani_de_tickets
in_books_22_comb['ticket_number'] = Beliani_de_tickets['Text'].str.extract(r'ticket(\d+)')
in_books_22_comb = in_books_22_comb.dropna(subset=["ticket_number"]) 

C:\Users\User\AppData\Local\Temp\ipykernel_14072\4052672927.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_books_22_comb['ticket_number'] = Beliani_de_tickets['Text'].str.extract(r'ticket(\d+)')


In [66]:
# all tickets numbers that need auction number to be added
all_ticket = pd.concat([in_books_22_split, in_books_22_comb])

In [52]:
tickets = all_ticket[all_ticket['ticket_number'].str.len() == 6]
invoices = all_ticket[all_ticket['ticket_number'].str.len() > 6]
invoices.rename(columns={'ticket_number':'invoice_number'},inplace=True)


C:\Users\User\AppData\Local\Temp\ipykernel_14072\3126770460.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invoices.rename(columns={'ticket_number':'invoice_number'},inplace=True)


In [55]:
invoices['invoice_number'] = invoices['invoice_number'].astype(str)
invoices['invoice_number'] = invoices['invoice_number'].astype(str).replace(r'\.0', '', regex=True)

C:\Users\User\AppData\Local\Temp\ipykernel_14072\31640403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invoices['invoice_number'] = invoices['invoice_number'].astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_14072\31640403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invoices['invoice_number'] = invoices['invoice_number'].astype(str).replace(r'\.0', '', regex=True)


In [56]:
invoices_auctions = pd.merge(invoices, auctions, on='invoice_number', how='left')

In [67]:
invoices_auctions['ticket_number'] = invoices_auctions['Text'].str.extract(r'ticket (\d+)')


In [70]:
# Run MySQL query to fetch tickets.
# Putty must be running while running the script
with open(db_credentials_dir + 'db_credentials.txt', 'r') as f:
    db_credentials = f.read().splitlines()

try:
    db_connections = connection.connect(
        host=db_credentials[0],
        user=db_credentials[1],
        password=db_credentials[2],
        database=db_credentials[3]
    )
    tickets_sql = open(file_dir + 'tickets_sql.sql')
    tickets_sql_query = tickets_sql.read()
    all_tickets_sql = pd.read_sql(tickets_sql_query,db_connections)

    # close the connection
    db_connections.close()  
except Exception as e:
    db_connections.close()
    print(str(e))

C:\Users\User\AppData\Local\Temp\ipykernel_14072\1176604031.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_tickets_sql = pd.read_sql(tickets_sql_query,db_connections)


In [73]:
tickets['ticket_number'] = tickets['ticket_number'].astype(str)
tickets['ticket_number'] = tickets['ticket_number'].astype(str).replace(r'\.0', '', regex=True)
all_tickets_sql['ticket_number'] = all_tickets_sql['ticket_number'].astype(str)
all_tickets_sql['ticket_number'] = all_tickets_sql['ticket_number'].astype(str).replace(r'\.0', '', regex=True)

C:\Users\User\AppData\Local\Temp\ipykernel_14072\2330440586.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickets['ticket_number'] = tickets['ticket_number'].astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_14072\2330440586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickets['ticket_number'] = tickets['ticket_number'].astype(str).replace(r'\.0', '', regex=True)


In [78]:
tickets_auctions = pd.merge(tickets, all_tickets_sql, on='ticket_number', how='left')

In [83]:
not_found = Beliani_de_tickets[~Beliani_de_tickets['Text'].isin(all_ticket['Text'])]

In [90]:
ticket_union = pd.concat([tickets_auctions, invoices_auctions, not_found])

In [8]:
Beliani_de_auctions_all = Beliani_de[Beliani_de['Text'].str.contains('auction', na=False) | Beliani_de['Text'].str.contains('auftrag', na=False) ]


In [94]:
ticket_union['type_split'] = 'ticket'

In [97]:
Beliani_de_auctions_all['auction_number'] = Beliani_de_auctions_all['Text'].str.extract(r'(?:auction|auftrag) (\d+)')


C:\Users\User\AppData\Local\Temp\ipykernel_14072\1105257916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Beliani_de_auctions_all['auction_number'] = Beliani_de_auctions_all['Text'].str.extract(r'(?:auction|auftrag) (\d+)')


In [101]:
Beliani_de_auctions_all['type_split'] = 'auction'

C:\Users\User\AppData\Local\Temp\ipykernel_14072\2369301859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Beliani_de_auctions_all['type_split'] = 'auction'


In [103]:
all_books_22 = pd.concat([Beliani_de_auctions_all, ticket_union])

In [109]:
all_books_22.rename(columns={'Amount_Netto(local)2':'Amount_Netto(local)'},inplace=True)

In [111]:
all_books_22['auction_number'] = all_books_22['auction_number'].astype(str)
all_books_22['auction_number'] = all_books_22['auction_number'].astype(str).replace(r'\.0', '', regex=True)
all_books_22['auction_number'] = all_books_22['auction_number'].str.strip()

In [112]:
FACT_all_transactions = pd.read_csv(file_dir + 'FACT_all_transactions.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_14072\3746988900.py:1: DtypeWarning: Columns (2,14) have mixed types. Specify dtype option on import or set low_memory=False.
  FACT_all_transactions = pd.read_csv(file_dir + 'FACT_all_transactions.csv')


In [113]:
FACT_all_transactions[['auction_number', 'txnid']] = FACT_all_transactions['Main auction'].str.split('_', expand=True)

In [114]:
FACT_all_transactions['Transaction_Date'] = pd.to_datetime(FACT_all_transactions['Transaction_Date'])

In [115]:
FACT_all_transactions['Selling_account'] = FACT_all_transactions['Selling_account'].astype(str)
FACT_all_transactions['Selling_account'] = FACT_all_transactions['Selling_account'].astype(str).replace(r'\.0', '', regex=True)

In [116]:
FACT_all_transactions['auction_number'] = FACT_all_transactions['auction_number'].astype(str)
FACT_all_transactions['auction_number'] = FACT_all_transactions['auction_number'].astype(str).replace(r'\.0', '', regex=True)
FACT_all_transactions['auction_number'] = FACT_all_transactions['auction_number'].str.strip()

In [117]:
FACT_all_transactions_2022= FACT_all_transactions[(FACT_all_transactions['Transaction_Date'] >= pd.Timestamp('2022-01-01')) & (FACT_all_transactions['Transaction_Date'] < pd.Timestamp('2023-01-01')) ]

In [119]:
DIM_all_auction = pd.read_csv(file_dir + 'DIM_all_auction.csv')

In [120]:
DIM_all_auction[['auction_number', 'txnid']] = DIM_all_auction['MAU_Main auction'].str.split('_', expand=True)


In [121]:
DIM_Company = DIM_all_auction[['auction_number', 'Company', 'Seller']]

In [122]:
DIM_Company.drop_duplicates(subset='auction_number', inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_14072\1269495965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DIM_Company.drop_duplicates(subset='auction_number', inplace=True)


In [123]:
DIM_Company['auction_number'] = DIM_Company['auction_number'].astype(str)
DIM_Company['auction_number'] = DIM_Company['auction_number'].astype(str).replace(r'\.0', '', regex=True)
DIM_Company['auction_number'] = DIM_Company['auction_number'].str.strip()

FACT_all_transactions_2022['auction_number'] = FACT_all_transactions_2022['auction_number'].astype(str)
FACT_all_transactions_2022['auction_number'] = FACT_all_transactions_2022['auction_number'].astype(str).replace(r'\.0', '', regex=True)
FACT_all_transactions_2022['auction_number'] = FACT_all_transactions_2022['auction_number'].str.strip()


C:\Users\User\AppData\Local\Temp\ipykernel_14072\3941791682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DIM_Company['auction_number'] = DIM_Company['auction_number'].astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_14072\3941791682.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DIM_Company['auction_number'] = DIM_Company['auction_number'].astype(str).replace(r'\.0', '', regex=True)
C:\Users\User\AppData\Local\Temp\ipykernel_14072\3941791682.py:3: SettingWithCopyWarning: 
A value is trying

In [124]:
transactions_2022_company = pd.merge(FACT_all_transactions_2022, DIM_Company, on='auction_number', how='left')

In [125]:
transactions_2022_company['auction_number'] = transactions_2022_company['auction_number'].astype(str)
transactions_2022_company['auction_number'] = transactions_2022_company['auction_number'].astype(str).replace(r'\.0', '', regex=True)
transactions_2022_company['auction_number'] = transactions_2022_company['auction_number'].str.strip()

In [126]:
in_bi22 = transactions_2022_company[~transactions_2022_company['auction_number'].isin(all_books_22['auction_number'])]

In [128]:
in_bi22.to_csv(file_dir + 'in_bi22_fixed.csv', index=False, encoding='utf-8')